In [ ]:
import pandas as pd
import pycountry
import numpy as np

In [ ]:
PATH_HAPPY = "../Data/world-happiness-report.csv"
PATH_HAPPY_2021 = "../Data/world-happiness-report-2021.csv"
PATH_ALCOHOL = "../Data/alcohol-consumption.csv"

# Data Processing

In [ ]:
df_happy_2021 = pd.read_csv(PATH_HAPPY_2021)
df_happy_2021.head()

In [ ]:
df_happy = pd.read_csv(PATH_HAPPY)
df_happy

### Happiness data concatenation

Let's take a look at the range of the historical data.

In [ ]:
df_happy["year"].describe()

Historical data goes from 2005 to 2020. 

We will first harominze the names of the columns in the 2021 set, in order to be ablte to merge them easily with the historical data.

In [ ]:
# let's include the year in the current dataset
df_happy_2021["year"] = 2021

In [ ]:
rename_columns = {'Ladder score':'Life Ladder', 'Logged GDP per capita':'Log GDP per capita',
'Healthy life expectancy':'Healthy life expectancy at birth'}

In [ ]:
df_happy_2021 = df_happy_2021.rename(columns = rename_columns)

Let's now concatenate the two sets. For consistency reasons, we'll take only the columns already present into the historical set and the 2021 set. Note that in this case, this does not exclude any observed statistics, only those columns which were computed directly from the others. 

In [ ]:
df = pd.concat([df_happy, df_happy_2021], join="inner", ignore_index=True)

In [ ]:
df

### Adding country codes 

To ensure consistency and completeness between both our datasets and external datasets further used for visualization,
we will add country codes following the ISO standards and rename the countries according to those same ISO standards.

In [ ]:
# We built here a list of country names that were not standard and their standard conversion. 
names_conversion = {"Czech Republic": "Czechia", 
                    "Taiwan Province of China": "Taiwan, Province of China", 
                    "South Korea": "Korea, Republic of", 
                    "Moldova": "Moldova, Republic of", 
                    "Bolivia": "Bolivia, Plurinational State of", 
                    "Russia" : "Russian Federation", 
                    "Hong Kong S.A.R. of China": "Hong Kong", 
                    "Vietnam": "Viet Nam", 
                    "Congo (Brazzaville)": "Congo",
                    "Congo (Kinshasa)": "Congo, The Democratic Republic of the",
                    "Ivory Coast": "Côte d'Ivoire",
                    "Laos": "Lao People's Democratic Republic", 
                    "Venezuela": "Venezuela, Bolivarian Republic of",
                    "Iran": "Iran, Islamic Republic of", 
                    "Palestinian Territories": "Palestine, State of", 
                    "Swaziland": "Eswatini",
                    "Syria": "Syrian Arab Republic",
                    "Tanzania": "Tanzania, United Republic of"}

In [ ]:
df["Country name"] = df["Country name"].replace(names_conversion)

We will use the pycountry library and retrieve the iso_2 and iso_3 codes for country names. 

In [ ]:
input_countries = [a for a in df["Country name"].to_numpy()]
countries_2 = {}
countries_3 = {}
for country in pycountry.countries:
    countries_2[country.name] = country.alpha_2
    countries_3[country.name] = country.alpha_3
    
codes_2 = [countries_2.get(country, 'Unknown code') for country in input_countries]
codes_3 = [countries_3.get(country, 'Unknown code') for country in input_countries]

df["iso_2"] = codes_2
df["iso_3"] = codes_3

In [ ]:
df[df['iso_3'] == "Unknown code"]

Kosovo and North Cyprus don't benefit of full international recognition and do not have official country codes. 
However, for completion reasons, we will manually input their temporary country codes.
Regarding Somaliland region, most of the data being NaN, we will simply exclude it from our dataset. 

In [ ]:
# some iso standards are not yet define. We enter by hand the temporary ones. 
df.loc[df["Country name"] == "Kosovo", "iso_2"] = "XK"
df.loc[df["Country name"] == "Kosovo", "iso_3"] = "XKX"
df.loc[df["Country name"] == "North Cyprus", "iso_2"] = "CTR"
df.loc[df["Country name"] == "North Cyprus", "iso_3"] = "CTR"

In [ ]:
df = df[df["Country name"] != "Somaliland region"]

In [ ]:
df[df['iso_3'] == "Unknown code"]

Therefore now, all our country codes appear to be included. This will tremendously facilitate 
join operations with other sets. 

In [ ]:
df.head()

Let's take a look at the completeness of this dataset. 

In [ ]:
df.groupby(['Country name'])['Country name'].count()

We clearly see some years are missing for some data. Let's solve that by adding NaN values lines inbetween. 

In [ ]:
# we set an index as "iso_3/Year"
df.set_index(['iso_3', 'year'], inplace=True)
df

In [ ]:
# then we reconstruct a combination of all indices and reindex
index = pd.MultiIndex.from_product(df.index.levels)
df = df.reindex(index)

### Including alcohol consumption data

In [ ]:
df_alcohol = pd.read_csv(PATH_ALCOHOL)

In [ ]:
df_alcohol

In [ ]:
df_alcohol["Year"].describe()

Regarding alcohol consumption, our data ranges from 2005 to 2018. However, a quick eyeball shows the data as being relatively incomplete (Afghanistan for example, only contains 2010, 2015 and 2018 data). Once our data will be merged with the happiness dataframe, we will have NaN values for many years. However we should be able to easily interpolate, using the prebuilt `df.interpolate()` function. 

In [ ]:
# we do define the same index as the previous set, to facilitate the Join operation
df_alcohol = df_alcohol.rename(columns = {'Year':'year', 'Code':'iso_3', "Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)": "Alcohol consumption"})
df_alcohol.set_index(['year', 'iso_3'], inplace=True)

In [ ]:
df_alcohol

In [ ]:
df = df.merge(right = df_alcohol, how = "left", on = ["iso_3", "year"])
df

### Interpolate

We will interpolate all the possible values and fill the gaps with the closest values, linearly. Note that for countries where only NaN values are available 

In [ ]:
df = df.groupby("iso_3").apply(
    lambda group: group.interpolate(method = "linear", limit_direction = 
                                    "both").fillna(method = "ffill").fillna(method = "bfill"))

### Outliers

In order to identify outliers, we will use an "outlier rate", meaning the ratio of `Life Ladder/Log GDP per capita`. The higher the ratio, the happier the country compared to what it should be based on its inhabitants financial ressources alone. 

In [ ]:
df["Happiness/GDP cap."] = df["Life Ladder"]/df["Log GDP per capita"]
df

As a quick view, our main outliers are shown below. 

In [ ]:
df.sort_values("Happiness/GDP cap.", ascending = False).head()

# Export 

Now we will export those data year by year in a csv format. The goal is to be easily 
reusable without further computation in our website.

In [ ]:
export_path = "../Data/Processed/"

In [ ]:
### for each year, we put all the information in a different csv file 
for year in df.index.levels[1]:
    df[np.in1d(df.index.get_level_values(1), [year])].to_csv(export_path+"data_"+str(year)+".csv") 